### Load libraries and set file names

In [1]:
## Parameters specific to where your folders are and your data
parameter_file = 'params/params.yaml'
import yaml
import sys

with open(parameter_file,'r') as f:
    doc = yaml.load(f)

#p = dic2obj(**doc)

data_folder = doc['data_folder']
tissues = doc['tissues'].split(',')
sys.path.append(doc['pipeline_path'])
ref_fa = doc['ref_fa']
anno_gff=doc['annotation']


tss_annotation = doc['tss_annotation']

import os
os.chdir
import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
import tqdm

from os.path import basename
##mpl.use('Agg')
#mpl.style.use('ggplot')
#mpl.style.use('fivethirtyeight')
from Homer import *

print('Number of tissues: ',len(tissues))

('Number of tissues: ', 15)


In [2]:
sys.path.append("/home/isshamie/software/homebrew/parallel_functions/")
import parallel_functions as pf

In [3]:
import motif_analysis

In [4]:
motif_folder = doc['motif_folder']
motifs_of_interest = map(lambda x: os.path.basename(x).split('.')[0],glob.glob(motif_folder + '/*motif'))
motifs_of_interest

['scl',
 'cdx2',
 'fli',
 'pu1',
 'elf3',
 'klf10',
 'sp5',
 'oct2',
 'ebna1',
 'phox2a',
 'hinfp',
 'bcl6',
 'smad4',
 'pknox1',
 'gata3',
 'myf5',
 'staf',
 'pbx2',
 'tbet',
 'znf528',
 'pax7-longest',
 'spdef',
 'tra',
 'klf14',
 'nrf',
 'p53-myc',
 'gata3',
 'sp2',
 'foxa1',
 'hif2a',
 'vdr',
 'znf264',
 'gabp',
 'tead',
 'irf1',
 'sf1',
 'znf136',
 'p65',
 'klf6',
 'stat6',
 'tcf7l2',
 'zscan22',
 'six4',
 'gfi1b',
 'foxl2',
 'tr4',
 'gata3',
 'mef2d',
 'grhl2',
 'ptf1a',
 'znf322',
 'lxre',
 'ar-half',
 'npas',
 'fosl2',
 'arnt',
 'gata3',
 'lhx2',
 'nfat-ap1',
 'ets-runx',
 'erg',
 'meis1',
 'tbx20',
 'znf675',
 'hoxd12',
 'nfatc1',
 'rfx2',
 'znf415',
 'hnf1b',
 'e2f6',
 'usf2',
 'foxo3',
 'pit1-long',
 'ews-fli1',
 'oct',
 'ehf',
 'are-fox',
 'pax3-fkhr',
 'jun-cre',
 'sox3',
 'limb',
 'runx1m',
 'tfe3',
 'stat1',
 'couptf2',
 'heb',
 'zfp809',
 'nfy',
 'gata2',
 'p53',
 'zfp281',
 'brn2',
 'jun-ap1',
 'elf1',
 'rxr',
 'atf7',
 'zic3',
 'foxa2',
 'pax5',
 'pax7-long',
 'znf416

In [ ]:
fa_file = 'Results/merged/samples.merge.anno.fa'

# Run funtions

### 1. Get the score and instances for a set of 4 motifs
(It takes longer to run the instances)

In [ ]:
motifs_of_interest = ['tata', 'sp1', 'ctcf', 'inr']
pf.parallel_ar(
    ar=np.array(motifs_of_interest),
    func=motif_analysis.all_motifs_of_interest_score,
    func_args=(fa_file, motif_folder, 'Results/motifs/allPeaks_score'),
    num_processes=4)

sp1
ctcf
inr
tata


484058it [00:54, 8891.82it/s]
484058it [01:13, 6596.29it/s]
484058it [01:13, 6592.83it/s]
484058it [01:49, 4416.01it/s]


Time taken: 121


In [ ]:
motifs_of_interest = ['tata', 'sp1', 'ctcf', 'inr']
pf.parallel_ar(
    ar=np.array(motifs_of_interest),
    func=motif_analysis.all_motifs_of_interest_instance,
    func_args=(fa_file, motif_folder, 'Results/motifs/allPeaks_instance'),
    num_processes=4)

### Run over entire motifs_of_interest to calculate the score

In [ ]:
motif_folder = doc['motif_folder']
motifs_of_interest = map(lambda x: os.path.basename(x).split('.')[0],glob.glob(motif_folder + '/*motif'))
pf.parallel_ar(
    ar=np.array(motifs_of_interest),
    func=motif_analysis.all_motifs_of_interest_score,
    func_args=(fa_file, motif_folder, 'Results/motifs/allPeaks_score'),
    num_processes=24)

## There are 484,059 peaks. 
## Takes ~40 minutes for each motif file to find the instance
## 393 motif files. 

40*393/60=262 hours!  
Can parallelize to 16 cores, which gives 262/16=17 hours. This is still a bit much  
One simplification: Only observe peaks <40kb from the tss

In [ ]:
peaks = pd.read_csv("/data/isshamie/TSS/Analysis/10_15_Results/Results/merged/peaks_with_tss_distances.tsv",sep='\t',index_col=0)
peaks[np.abs(peaks['Distance to TSS'])<5000]

### Only use peaks <40kb from start site

In [ ]:
## Load in peak file


In [ ]:
%%time
for i in motifs_of_interest:
    f_out =  'Results/motifs/duplicateSeparate_peaks_merged_anno' + i + '.p'
    fa_file = 'Results/merged/duplicateSeparate_peaks_merged_anno.fa'
    motif_f = motifs_folder + i + '.motif.pfm'
    wrap_fa_motif(fa_file,motif_f,f_save = f_out)